<a href="https://colab.research.google.com/github/Quikks1lver/WayFinder/blob/main/NodeXYExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import xml.etree.ElementTree as et
import json
import time

In [2]:
xml = files.upload()

Saving highways_2.osm to highways_2.osm


In [26]:
# Load in the osm XML file and get the root

mytree = et.parse("highways_2.osm")

myroot = mytree.getroot()

print(myroot.attrib)

{'version': '0.6', 'generator': 'Osmosis 0.48.3'}


In [27]:
# Map holding node information -- node id : w/lat, long
nodeInfoMap: dict = {}

# Stores temporarily the reference id for each node
nodeId: int = 0

# Stores temporarily long/lat for each node
longX: float = 0.0
latY: float = 0.0

# Map to hold lat/long for nodes, temporarily
nodeGeolocation = {}

# Map to hold way info for the nodes
wayInfoForNodeMap: dict = {}

In [28]:
# create a hash map to store way : node info
numNodes: int = 0

for node in myroot.findall("node"):
  longX = node.attrib["lon"]
  latY = node.attrib["lat"]

  nodeGeolocation = {}
  nodeGeolocation["longX"] = longX
  nodeGeolocation["latY"] = latY

  nodeInfoMap[int(node.attrib["id"])] = nodeGeolocation

  numNodes += 1

In [29]:
# print(nodeInfoMap)
print(numNodes)

4425


In [30]:
# Declare necessary vars
count: int = 0
DEBUGGING: bool = False
TEST_X_NODES: bool = False
successfullyFoundStreet: bool = True
outputMap: dict = {} # Holds node id references : node information (id, x, y, list of ways the node is a part of, etc.)

# For adding in extra nodes
ADD_IN_EXTRA_NODES: bool = True # toggle t/f for your purposes
extraNodesInBetweenCount: int = 500
evenOdd: int = 1
firstNodeLongX: float = 0.0
firstNodeLatY: float = 0.0
secondNodeLongX: float = 0.0
secondNodeLatY: float = 0.0
newNodeLongX: float = 0.0
newNodeLatY: float = 0.0
outputMap2: dict = {}

# for timing the code
t0 = time.time()

# loop through all nodes
for way in myroot.findall("way"):
  count += 1

  if count > 5 and TEST_X_NODES:
    break

  try:    
    wayInfoForNodeMap = {}
    
    # populate id for way in wayinfo map
    wayInfoForNodeMap["wayId"] = int(way.attrib["id"])
    
    successfullyFoundStreet = False
    # populate name for way in the wayinfo map
    for childTag in way.findall("tag"):
      if childTag.attrib['k'] == "name":
        wayInfoForNodeMap["streetName"] = childTag.attrib["v"]
        successfullyFoundStreet = True
        break
      else:
        continue
      childCount += 1

    if not successfullyFoundStreet:
      continue

    # draw relationships between ways <=> nodes to add into output map
    for node in way.findall("nd"):        
        nodeId = int(node.attrib["ref"])
        
        if int(nodeId) not in outputMap.keys():
          longX = nodeInfoMap.get(nodeId)["longX"]
          latY = nodeInfoMap.get(nodeId)["latY"]
          tempWayInfo = []
          tempWayInfo.append(wayInfoForNodeMap["streetName"])
          outputMap[int(nodeId)] = {"id": int(nodeId), "longX": float(longX), "latY": float(latY), "waysInfo": tempWayInfo}
        else:
          outputMap[int(nodeId)]["waysInfo"].append(wayInfoForNodeMap["streetName"])

  except:
    continue

t1 = time.time()

print("============================================================")
print(f"Code took {t1 - t0} to run. Ran through {count} ways")
print("============================================================")

Code took 0.019908428192138672 to run. Ran through 659 ways


In [31]:
len(outputMap)

4129

In [32]:
# For adding in extra nodes

firstNodeLongX, firstNodeLatY, secondNodeLongX, secondNodeLatY, newNodeLongX, newNodeLatY = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
evenOdd = 1
for way in myroot.findall("way"):
  for node in way.findall("nd"):
    nodeId = int(node.attrib["ref"])

    if ADD_IN_EXTRA_NODES == True:
      # print(extraNodesInBetweenCount)
      if evenOdd == 1:
        firstNodeLongX = nodeInfoMap.get(nodeId)["longX"]
        firstNodeLatY = nodeInfoMap.get(nodeId)["latY"]
        evenOdd = 2

      elif evenOdd == 2:
        secondNodeLongX = nodeInfoMap.get(nodeId)["longX"]
        secondNodeLatY = nodeInfoMap.get(nodeId)["latY"]
        x = (float(firstNodeLongX) + float(secondNodeLongX)) / 2
        y = (float(firstNodeLatY) + float(secondNodeLatY)) / 2

        struct = {"id": extraNodesInBetweenCount, "longX": x, "latY": y, "waysInfo": wayInfoForNodeMap["streetName"]}
        outputMap2[extraNodesInBetweenCount] = struct

        evenOdd = 1
      
      extraNodesInBetweenCount += 1

In [33]:
len(outputMap2)

2675

In [34]:
outputMap.update(outputMap2)
print(len(outputMap))

6804


In [35]:
# the json file where the output must be stored 

with open("highways_new.json", "w") as outputFile:
  json.dump(outputMap, outputFile)